<a href="https://colab.research.google.com/github/SH0312/KOSA_Pytorch/blob/main/1_Linear_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network
- MNIST data
- only fully connected layers

## 1. Settings
### 1) Import required libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Set hyperparameters

In [3]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

## 3. Data Generation

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[1], mnist_train.__len__())
mnist_test.__getitem__(0)[1], mnist_test.__len__()

5 60000


(7, 10000)

### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

## 4. Model & Optimizer

### 1) CNN Model

In [7]:
class Linear(nn.Module):
    def __init__(self):
        super(Linear,self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(784,300),
            nn.ReLU(),
            nn.Linear(300,100),
            nn.ReLU(),
            nn.Linear(100,10),
            nn.ReLU()
        )       
        
    def forward(self,x):
        out = x.view(batch_size,-1)
        out = self.layer(out)

        return out

model = Linear().cuda()

### 2) Loss func & Optimizer

In [8]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 5. Train 

In [9]:
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = Variable(image).cuda()
        y_= Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)

tensor(2.3107, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3117, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3020, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2912, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2900, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3003, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2806, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3169, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2989, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2792, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2809, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2815, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2673, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2474, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2947, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2668, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2908, device='cuda:0', grad_fn=<NllLossBackward

## 6. Test

In [10]:
correct = 0
total = 0

for image,label in test_loader:
    x = Variable(image,volatile=True).cuda()
    y_= Variable(label).cuda()

    output = model.forward(x)
    _,output_index = torch.max(output,1)
        
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print("Accuracy of Test Data: {}".format(100*correct/total))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data: 45.119998931884766
